<a href="https://colab.research.google.com/github/MeidanGR/GooglePlayStoreAnalysis/blob/main/GooglePlayStore_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Play Store Data Preprocessing**
This data preprocessing progress has been established for an academic project of a BI course. The data will be analyzed within the project.

Major thanks to **Gautham Prakash** for publishing & maintenance this dataset, which you can find at https://www.kaggle.com/gauthamp10/google-playstore-apps


# **Google auth & libraries**

In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
import pandas as pd
import numpy as np

# **CSV LOAD**


In [93]:
#Loading csv
df = pd.read_csv('/content/drive/My Drive/Google-Playstore.csv')

print("Dataframe original shape is", np.shape(df))
df.sample(2)

Dataframe original shape is  (603047, 23)


,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Currency,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
591716,Women Handbag Ideas,com.starmobileapps.modernhandbagideaswomen,Lifestyle,4.1,19.0,"10,000+",10000.0,18898,True,0.0,USD,6.8M,4.0 and up,Star Mobile Apps,NaN,starmobileapps1@gmail.com,"Nov 21, 2017",25/10/2019 15:05,Everyone,https://starmobileapps.blogspot.in/p/privacy-p...,True,False,False
14351,বাচ্চাদের ছোট সূরা বাংলা - Small Surah for Kids,com.sevenonelab.bangla_small_surah_for_kids_ea...,Education,4.5,38.0,"10,000+",10000.0,15539,True,0.0,USD,100M,4.1 and up,71 lab,NaN,amit0167604@gmail.com,"Jul 13, 2019",04/04/2020 16:34,Everyone,https://71lab.blogspot.com/2018/09/privacy-pol...,True,False,False


# **DATA FILTERING**
## Agenda: 
Data reduction up to ~100K rows, for a decent memory space & faster analysis.

## Guidelines:
- **Minimum Android** is the minimum android version which allowes operation for the app. The mose popular values were found: 4.1\4.0.3\6.0 and up & Varies with device.
- **Size** should exceed 1MB.
- **Currency** will be USD only.
- **Last updated** during 2020.


In [94]:
#Coloumn labels space fix
df.columns = [x.replace(" ", "_") for x in df.columns]

#Rows SELECT
df = df.query('(Minimum_Android == "4.1 and up" | Minimum_Android == "4.0.3 and up" | Minimum_Android == "Varies with device" | \
Minimum_Android == "6.0 and up") & not Size.str.endswith("k") & Currency == "USD" & Last_Updated.str.contains("2020")',engine='python')

#Coloumns FILTER
df = df.filter(items=['App_Name', 'Category', 'Rating', 'Rating_Count', 'Installs', 'Price', 'Size','Released','Last_Updated','Ad_Supported', 'In_App_Purchases'])

#Saving a copy of dataframe
dfcopy = df.copy(deep=True)

print("Dataframe reduced shape is", np.shape(df))
df.sample(10)



Dataframe reduced shape is (112092, 11)


,App_Name,Category,Rating,Rating_Count,Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
159759,DinStats,Productivity,0.0,0.0,50+,14.99,2.9M,"Oct 1, 2019",05/07/2020 13:43,False,False
215096,Folder Video Player,Video Players & Editors,4.0,4471.0,"500,000+",0.00,27M,"Jan 29, 2014",14/06/2020 16:43,True,False
124960,Colony Bank Business Mobile,Finance,0.0,0.0,500+,0.00,9.3M,"Apr 27, 2015",08/04/2020 6:54,False,False
572381,Vehicle RTO registration information,Business,4.2,748.0,"100,000+",0.00,9.4M,"May 10, 2017",10/03/2020 21:29,True,False
83453,Bipi - Monthly subscriptions to cars,Travel & Local,3.7,411.0,"50,000+",0.00,14M,"Mar 22, 2017",03/01/2020 17:13,False,False
592996,Word Search Pop - Free Fun Find & Link Brain G...,Word,4.7,7754.0,"500,000+",0.00,Varies with device,"Oct 27, 2018",08/07/2020 14:52,True,True
405406,"Olyfox Browser-Fast & Secure Browser, Safe Bro...",Communication,4.2,187.0,"10,000+",0.00,5.2M,"Jul 3, 2018",07/03/2020 17:29,True,False
468103,Robin by Serieplay,Role Playing,4.2,761.0,"10,000+",0.00,70M,"Mar 14, 2020",24/06/2020 16:03,True,True
151128,Dating app for free: dating & chat - Love.ru,Dating,4.0,8143.0,"1,000,000+",0.00,2.7M,"Jun 9, 2014",19/05/2020 12:29,True,True
592820,Word Picture Fun Quiz Game for Indians,Word,4.8,5.0,500+,0.00,Varies with device,"Mar 27, 2020",18/04/2020 20:02,True,False


# **INITIAL DATA INFO**

In [79]:
print("Dataframe NULL count:")
print(df.isnull().sum(axis = 0), end="\n\n")

df.info()

Dataframe NULL count:
App_Name               1
Category               0
Rating              1880
Rating_Count        1880
Installs               0
Price                  0
Size                   0
Released            6338
Last_Updated           0
Ad_Supported           0
In_App_Purchases       0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112092 entries, 20 to 603046
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App_Name          112091 non-null  object 
 1   Category          112092 non-null  object 
 2   Rating            110212 non-null  float64
 3   Rating_Count      110212 non-null  float64
 4   Installs          112092 non-null  object 
 5   Price             112092 non-null  float64
 6   Size              112092 non-null  object 
 7   Released          105754 non-null  object 
 8   Last_Updated      112092 non-null  object 
 9   Ad_Supported      112092 non-null  bool   
 

In [36]:
#Retrieve coloumn general info 
name = 'Category'
df[name].describe()

count         97206
unique           48
top       Education
freq          11064
Name: Category, dtype: object

# **DATA CLEAN & CORRECTION**
- Null values that were found in **App_Name, Released** are not acceptable for furthor analysis.
- The null values in **Rating, Rating_Count** will be replaced with mean values.

In [96]:
#NULL Extraction
df = df.query('not Released.isnull() & not Rating.isnull() & not Rating_Count.isnull() & not App_Name.isnull()',engine='python')

# Function replaces np.nan values in all of the data frame with relevant statistics \ dal7collab
def rep_missings(data_frame):
    d = data_frame.copy(deep = True)
    for c in data_frame:
        if (data_frame[c].dtype == 'float64'):
            if data_frame[c].isnull().values.any() :
                m = data_frame[c].dropna().mean()
                d[c].fillna(m, inplace = True)
    return d;
# Activating the function
df = rep_missings(df)

#Coloumn types correction
df['App_Name'] = df['App_Name'].astype({'App_Name': 'string'})
df['Category'] = df['Category'].astype({'Category': 'string'})
df['Rating_Count'] = df['Rating_Count'].astype({'Rating_Count': 'int32'})
df['Installs'] = df['Installs'].astype({'Installs': 'string'})
df['Size'] = df['Size'].astype({'Size': 'string'})
df['Released']= pd.to_datetime(df['Released']) 
df['Last_Updated']= pd.to_datetime(df['Last_Updated']) 

#'Size' coloumn excluding "Varies with device"
df['Size'] = df['Size'].replace(to_replace="Varies with device", 
                                   value = np.nan)


print("Dataframe final shape is", np.shape(df))
df.sample(10)


Dataframe reduced shape is (105746, 11)


,App_Name,Category,Rating,Rating_Count,Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
602684,Z-Road : Save the World from the Z Virus,Action,0.0,0,"1,000+",0.0,<NA>,2020-04-05,2020-01-04 10:47:00,True,False
180360,고양버스 스마트,Maps & Navigation,3.1,108,"10,000+",0.0,22M,2014-06-03,2020-07-17 10:50:00,True,False
568391,US Dollar x New Israeli Shekel,Finance,4.0,16,"5,000+",0.0,9.2M,2016-12-11,2020-05-30 20:11:00,True,False
311938,Lagu BTOB Lengkap Offline,Music & Audio,0.0,0,500+,0.0,37M,2019-02-08,2020-11-04 08:21:00,True,False
173978,🗺️💄Girl Dress Up Salon - World Travel,Casual,4.2,1423,"100,000+",0.0,21M,2017-03-16,2020-07-13 14:32:00,True,False
404053,offline French dictionary,Education,4.1,24,"5,000+",0.0,5.2M,2019-03-28,2020-03-17 23:51:00,True,False
528469,Tap Tap Trillionaire - Cash Clicker Adventure,Simulation,4.6,40827,"1,000,000+",0.0,8.0M,2016-05-17,2020-04-07 11:30:00,True,True
103917,Canossianas Mobile,Education,3.0,12,500+,0.0,16M,2017-10-05,2020-02-17 05:27:00,False,False
63171,Autumn Live Wallpaper HD,Personalization,3.4,5,500+,0.0,13M,2018-09-05,2020-07-07 16:52:00,True,False
359659,Monsterland 2. Physics puzzle game,Puzzle,4.2,1138,"100,000+",0.0,4.7M,2017-01-17,2020-04-06 10:35:00,True,True


# **POSTPROCESSING DATA INFO**

In [81]:
#Genereal information after preprocessing
print(df.info(),end = '\n\n')

### Data Frame general information: ###

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105746 entries, 81 to 603045
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   App_Name          105746 non-null  string        
 1   Category          105746 non-null  string        
 2   Rating            105746 non-null  float64       
 3   Rating_Count      105746 non-null  int32         
 4   Installs          105746 non-null  string        
 5   Price             105746 non-null  float64       
 6   Size              97206 non-null   string        
 7   Released          105746 non-null  datetime64[ns]
 8   Last_Updated      105746 non-null  datetime64[ns]
 9   Ad_Supported      105746 non-null  bool          
 10  In_App_Purchases  105746 non-null  bool          
dtypes: bool(2), datetime64[ns](2), float64(2), int32(1), string(4)
memory usage: 7.9 MB
None



,App_Name,Category,Rating,Rating_Count,Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
81,"""RS Station"" - Road side station navigator",Travel & Local,3.7,35,"5,000+",0.0,5.6M,2018-01-28,2020-11-03 09:17:00,False,False
111,#BeApp,Entertainment,4.0,390,"100,000+",0.0,51M,2020-04-17,2020-07-22 14:23:00,True,False
116,#CrockFit,Health & Fitness,4.8,187,"10,000+",0.0,1.0M,2019-01-05,2020-07-20 16:24:00,False,True
119,#БешенаяСушка,Health & Fitness,4.0,453,"50,000+",0.0,1.0M,2018-04-19,2020-05-24 16:57:00,False,False
153,#LikeU,Dating,2.7,69,"10,000+",0.0,1.0M,2019-05-29,2020-06-16 13:12:00,False,False
178,#SkyesSquad Gifs by Facemoji Emoji Keyboard,Personalization,4.6,253,"10,000+",0.0,1.0M,2017-11-21,2020-05-14 12:01:00,True,False
189,#The Hashtags Emoji Sticker With Funny Emotions,Personalization,4.7,301,"100,000+",0.0,1.0M,2017-09-01,2020-05-13 15:01:00,True,False
198,#walk15,Health & Fitness,3.8,549,"50,000+",0.0,1.0M,2018-03-11,2020-07-16 17:50:00,False,True
210,&frankly,Business,4.0,52,"10,000+",0.0,20M,2015-05-07,2020-04-30 14:15:00,False,False
215,( ͡° ͜ʖ ͡°) PugWars,Action,4.0,3626,"500,000+",0.0,1.0M,2018-10-24,2020-01-15 20:04:00,True,False


# **EXPORT TO GOOGLE DRIVE**


In [99]:
#Exporting a csv into google drive
df.to_csv('/content/drive/My Drive/GooglePlay-Store_Optimized.csv')